In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff  # 圖形工廠
from plotly.subplots import make_subplots  # 繪製子圖
import numpy as np
from Upload_git import upload
from os import system
import time
from time import ctime, sleep
from Upload_git import upload
import schedule
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from statsmodels.regression.rolling import RollingOLS, RollingWLS
from scipy import stats
import revenue_function as fn
import pickle

In [ ]:
def get_ratio(data):
    ttl_rev = data.rev.sum()/len(data)*12
    data['ratio'] = round(data.rev/ttl_rev, 4)
    return data


def get_avg_ratio(data):
    if len(data) >= 5:
        weight = [1, 1, 1, 3, 5]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) == 4:
        weight = [1, 1, 2, 3]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) == 3:
        weight = [1, 1, 2]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) <= 2:
        roll_num = len(data)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(data.ratio.mean(), 4)

    
def predict(data):
    a = data.groupby('Year', as_index=False).apply(get_ratio)
#     a.ratio.tail(1).values[0] = round(a.rev.tail(1)/a.rev.rolling(12).sum().tail(1), 4)
    ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
    predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(13)  # 僅向後預測一年
    predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
    predict['predict_month_rev'] = round(round(predict['predict_annul_rev'].mean(), 2) * predict.wt_avg_ratio)
    predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
    predict['rev'] = predict['predict_month_rev']
    predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
    predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
    predict = predict.groupby('month', as_index=False).apply(get_yoy).sort_values(['Year', 'month'])
    predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
    predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
    return predict.tail(12)
        
a.groupby('st_code', as_index=False).apply(predict).reset_index(drop=True).head(30)

In [4]:
"""
connect to db
"""
host='database-1.cyn7ldoposru.us-east-1.rds.amazonaws.com'
port='5432'
user='Yu'
password='m#WA12J#'
database="JQC_Revenue1"

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
        user, password, host, port, database), echo=True)
db = pd.read_sql('tej_revenue', engine).drop('index', axis=1)

profit = pd.read_excel('2015income_stat.xlsx')
profit['營業收入淨額'] = round(profit['營業收入淨額']/1000, 1)
profit['營業成本'] = round(profit['營業成本']/1000, 1)
profit['營業毛利'] = round(profit['營業毛利']/1000, 1)
profit['營業費用'] = round(profit['營業費用']/1000, 1)
profit['其他收益及費損淨額'] = round(profit['其他收益及費損淨額']/1000, 1)
profit['營業利益'] = round(profit['營業利益']/1000, 1)
profit['營業外收入及支出'] = round(profit['營業外收入及支出']/1000, 1)
profit['稅前淨利'] = round(profit['稅前淨利']/1000, 1)
profit['所得稅費用'] = round(profit['所得稅費用'], 1)
profit['其他綜合損益－OCI'] = round(profit['其他綜合損益－OCI']/1000, 1)
profit['所得稅費用'] = round(profit['所得稅費用']/1000, 1)
profit['稀釋稅後淨利'] = round(profit['稀釋稅後淨利']/1000, 1)
profit['歸屬非控制權益淨利（損）'] = round(profit['歸屬非控制權益淨利（損）']/1000, 1)
profit['稅後淨利est'] = profit['營業收入淨額'] - profit['營業成本'] - profit['營業費用'] + profit['其他收益及費損淨額']+ profit['營業外收入及支出'] - profit['所得稅費用'] - profit['歸屬非控制權益淨利（損）']
profit['eps_est'] = round(profit['稅後淨利est']/profit['加權平均股數']*1000, 2)
profit = profit.loc[:, ['代號', '名稱', '年/月', '加權平均股數', '營業收入淨額', '營業成本', '營業毛利', '營業費用',
                '其他收益及費損淨額', '營業利益', '營業外收入及支出', '稅前淨利', '所得稅費用', 
                '歸屬非控制權益淨利（損）', '稅後淨利est', '每股盈餘']]
profit = profit[(profit['營業成本'].isna() == False) & (profit['營業毛利'].isna() == False) & (profit['代號'] <= '9999')]

2023-03-03 09:20:32,735 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-03 09:20:32,741 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-03 09:20:33,157 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-03 09:20:33,158 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-03 09:20:33,566 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-03 09:20:33,567 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-03 09:20:33,984 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-03-03 09:20:33,986 INFO sqlalchemy.engine.Engine [generated in 0.00253s] {'name': 'tej_revenue'}
2023-03-03 09:20:34,615 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2023-03-03 09:20:34,617 INFO sqlalchemy.engine.Engine [generated i

2023-03-03 09:20:37,064 INFO sqlalchemy.engine.Engine [generated in 0.00305s] {'table_oid': 16512}
2023-03-03 09:20:37,269 INFO sqlalchemy.engine.Engine 
            SELECT
                pgd.description as table_comment
            FROM
                pg_catalog.pg_description pgd
            WHERE
                pgd.objsubid = 0 AND
                pgd.objoid = %(table_oid)s
        
2023-03-03 09:20:37,270 INFO sqlalchemy.engine.Engine [generated in 0.00152s] {'table_oid': 16512}
2023-03-03 09:20:37,690 INFO sqlalchemy.engine.Engine SELECT tej_revenue.index, tej_revenue.st_code, tej_revenue.st_name, tej_revenue.rev_period, tej_revenue.declaration_date, tej_revenue.rev, tej_revenue.declaration_year, tej_revenue.declaration_month, tej_revenue.new_industry_name, tej_revenue.minor_industry_name 
FROM tej_revenue
2023-03-03 09:20:37,691 INFO sqlalchemy.engine.Engine [generated in 0.00122s] {}


In [5]:
def get_yoy(data):
    data = data.sort_values('Year')
    data['yoy'] = round(data.rev.diff()/data.rev.shift(1), 4)
    return data

def get_st_mom_yoy(data):
    data['Year'] =  data.rev_period.astype(str).str[0:4]
    data['month'] = data.rev_period.astype(str).str[5:7]
    data['mom'] = round(data.rev.diff()/data.rev.shift(1), 4)
    yoy = pd.DataFrame()
    yoy = data.groupby('month').apply(get_yoy).reset_index(drop=True, inplace=False)
    return yoy#[yoy.Year >= '2016']

weight = [1, 1, 1, 3, 5]


def get_ratio(data):
    data['ratio'] = round(data.rev/data.rev.sum(), 4)
    return data


def get_avg_ratio(data):
    roll_num = len(weight)
    data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
    return round(np.average(data.ratio, weights = weight), 4)

def predict(data):
    try:
        a = data.groupby('Year', as_index=False).apply(get_ratio)
        ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
        predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(12)  # 僅向後預測一年
        predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
        predict['predict_month_rev'] = round(round(predict['predict_annul_rev'].mean(), 2) * predict.wt_avg_ratio)
        predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
        predict['rev'] = predict['predict_month_rev']
        predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
        predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
        predict = predict.groupby('month', as_index=False).apply(get_yoy).sort_values(['Year', 'month'])
        predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
        predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
        return predict.tail(12)
    except TypeError:
        pass

    
a = db.groupby('st_code', as_index=False).apply(get_st_mom_yoy).reset_index(drop=True)
predict_df = a.groupby('st_code', as_index=False).apply(predict).reset_index(drop=True)


# johnny 合併歷史跟預測資料，計算最新一期的表現是成長 or 衰退
johnny = pd.concat([a, predict_df], axis=0).sort_values(['st_code', 'Year', 'month'])


def growth(data):
    data['yoy年成長大於10'] = np.where(data.yoy >= 10, 1, 0)
    data['yoy成長'] = np.where(data.yoy > data.yoy.shift(1), 1, 0)
    data['yoy衰退'] = np.where(data.yoy <= data.yoy.shift(1), 1, 0)
    
    test = data.loc[:, ['yoy年成長大於10']]
    data['yoy近五月大於10'] = test.rolling(5).sum()
    data['yoy連續大於10'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    
    test = data.loc[:, ['yoy成長']]
    data['yoy近五月成長'] = test.rolling(5).sum()
    data['yoy連續成長'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    
    test = data.loc[:, ['yoy衰退']]
    data['yoy近五月衰退'] = test.rolling(5).sum()
    data['yoy連續衰退'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    return data



# 合併後計算檔案
concat = johnny.groupby('st_code', as_index=False).apply(growth).reset_index(drop=True)
concat

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,...,yoy,yoy年成長大於10,yoy成長,yoy衰退,yoy近五月大於10,yoy連續大於10,yoy近五月成長,yoy連續成長,yoy近五月衰退,yoy連續衰退
0,1101,台泥,2013/01,2013/02/18,9134465.0,2013,2,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
1,1101,台泥,2013/02,2013/03/08,5540346.0,2013,3,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
2,1101,台泥,2013/03,2013/04/10,9457971.0,2013,4,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
3,1101,台泥,2013/04,2013/05/10,9919269.0,2013,5,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
4,1101,台泥,2013/05,2013/06/10,9543782.0,2013,6,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212899,9962,有益,* 2023/09,2022/10/06,292786.0,2022,10,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,...,0.1079,0,1,0,0.0,0,3.0,1,2.0,0
212900,9962,有益,* 2023/10,2022/11/04,269637.0,2022,11,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,...,-0.1229,0,0,1,0.0,0,2.0,0,3.0,1
212901,9962,有益,* 2023/11,2022/12/07,336802.0,2022,12,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,...,0.0912,0,1,0,0.0,0,3.0,1,2.0,0
212902,9962,有益,* 2023/12,2023/01/09,291156.0,2023,1,M2000 鋼鐵工業,M20A6 不鏽鋼,2023,...,-0.2451,0,0,1,0.0,0,2.0,0,3.0,1


In [6]:
taifex_product = pd.read_html('https://www.taifex.com.tw/cht/5/stockMargining')[0].drop(['序號', '股票期貨標的證券'], axis=1)
taifex_product = taifex_product.rename({'股票期貨標的證券代號':'代號', '股票期貨  中文簡稱':'名稱'}, axis=1)
taifex_product['名稱'] = taifex_product['名稱'].str[:-2]
taifex_product
dividend = pd.read_excel('15_23股利.xlsx').rename({'盈餘分派_年度':'年/月'}, axis=1)

taifex_dividend = dividend[dividend['代號'].isin(taifex_product['代號']) == True]
taifex_profit = profit[profit['代號'].astype(int).isin(taifex_product['代號']) == True]
taifex_db = db[db.st_code.astype(int).isin(taifex_product['代號']) == True]
taifex_concat = concat[concat.st_code.astype(int).isin(taifex_product['代號']) == True]
taifex_concat

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,...,yoy,yoy年成長大於10,yoy成長,yoy衰退,yoy近五月大於10,yoy連續大於10,yoy近五月成長,yoy連續成長,yoy近五月衰退,yoy連續衰退
0,1101,台泥,2013/01,2013/02/18,9134465.0,2013,2,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
1,1101,台泥,2013/02,2013/03/08,5540346.0,2013,3,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
2,1101,台泥,2013/03,2013/04/10,9457971.0,2013,4,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
3,1101,台泥,2013/04,2013/05/10,9919269.0,2013,5,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,NaN,0,NaN,0,NaN,0
4,1101,台泥,2013/05,2013/06/10,9543782.0,2013,6,M1100 水泥工業,M11A 水泥製造,2013,...,NaN,0,0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211838,9945,潤泰新,* 2023/09,2022/10/11,2666937.0,2022,10,M9900 其他,M25A 建設,2023,...,0.0149,0,0,1,0.0,0,3.0,0,2.0,1
211839,9945,潤泰新,* 2023/10,2022/11/10,2309969.0,2022,11,M9900 其他,M25A 建設,2023,...,-0.0214,0,0,1,0.0,0,2.0,0,3.0,2
211840,9945,潤泰新,* 2023/11,2022/12/09,2658087.0,2022,12,M9900 其他,M25A 建設,2023,...,-0.0325,0,0,1,0.0,0,2.0,0,3.0,3
211841,9945,潤泰新,* 2023/12,2023/01/10,3392675.0,2023,1,M9900 其他,M25A 建設,2023,...,-0.3471,0,0,1,0.0,0,1.0,0,4.0,4


In [9]:
taifex_concat[taifex_concat.rev_period == '* 2023/02']

,st_code,st_name,rev_period,declaration_date,rev,declaration_year,declaration_month,new_industry_name,minor_industry_name,Year,...,yoy,yoy年成長大於10,yoy成長,yoy衰退,yoy近五月大於10,yoy連續大於10,yoy近五月成長,yoy連續成長,yoy近五月衰退,yoy連續衰退
121,1101,台泥,* 2023/02,2022/03/10,4562615.0,2022,3,M1100 水泥工業,M11A 水泥製造,2023,...,-0.1638,0,0,1,0.0,0,2.0,0,3.0,2
254,1102,亞泥,* 2023/02,2022/03/10,3220684.0,2022,3,M1100 水泥工業,M11A 水泥製造,2023,...,-0.2146,0,1,0,0.0,0,2.0,1,3.0,0
1318,1210,大成,* 2023/02,2022/03/09,6800021.0,2022,3,M1200 食品工業,M12B 油脂飼料,2023,...,-0.0953,0,0,1,0.0,0,1.0,0,4.0,4
1716,1216,統一,* 2023/02,2022/03/10,33064258.0,2022,3,M1200 食品工業,M12C 飲料乳品,2023,...,-0.0307,0,1,0,0.0,0,2.0,1,3.0,0
4276,1301,台塑,* 2023/02,2022/03/08,14982217.0,2022,3,M1300 塑膠工業,M13A1 泛用塑膠,2023,...,-0.2724,0,1,0,0.0,0,2.0,1,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207594,9904,寶成,* 2023/02,2022/03/10,18053812.0,2022,3,M9900 其他,M99K 鞋及旅行箱,2023,...,-0.0076,0,1,0,0.0,0,2.0,2,3.0,0
208645,9914,美利達,* 2023/02,2022/03/10,2580094.0,2022,3,M9900 其他,M99Z1 自行健行車,2023,...,0.1232,0,1,0,0.0,0,2.0,1,3.0,0
210902,9938,百和,* 2023/02,2022/03/07,1034889.0,2022,3,M9900 其他,M14Z 其他紡織,2023,...,-0.0666,0,1,0,0.0,0,2.0,1,3.0,0
211035,9939,宏全,* 2023/02,2022/03/10,1538779.0,2022,3,M9900 其他,M13B 塑膠製品,2023,...,0.0063,0,0,1,0.0,0,1.0,0,4.0,3


In [29]:
johnny[johnny.st_code == '8069'].loc[:,['st_code', 'st_name', 'rev_period', 'rev', 'yoy', 'mom']].tail(37)

,st_code,st_name,rev_period,rev,yoy,mom
172243,8069,元太,2021/01,1559034.0,0.5963,0.219500
172244,8069,元太,2021/02,1356295.0,1.0281,-0.130000
172245,8069,元太,2021/03,1520435.0,0.1909,0.121000
172246,8069,元太,2021/04,1428276.0,-0.0534,-0.060600
172247,8069,元太,2021/05,1237551.0,0.0462,-0.133500
172248,8069,元太,2021/06,1384590.0,0.3325,0.118800
172249,8069,元太,2021/07,1664313.0,0.1887,0.202000
172250,8069,元太,2021/08,861190.0,-0.4323,-0.482600
172251,8069,元太,2021/09,1871257.0,0.2192,1.172900
172252,8069,元太,2021/10,2206347.0,0.3819,0.179100
